Task IV: Quantum Generative Adversarial Network (QGAN)
You will explore how best to apply a quantum generative adversarial network (QGAN) to solve a High Energy Data analysis issue, more specifically, separating the signal events from the background events. You should use the Google Cirq and Tensorflow Quantum (TFQ) libraries for this task. 
A set of input samples (simulated with Delphes) is provided in NumPy NPZ format [Download Input]. In the input file, there are only 100 samples for training and 100 samples for testing so it won’t take much computing resources to accomplish this 
task. The signal events are labeled with 1 while the background events are labeled with 0. 
Be sure to show that you understand how to fine tune your machine learning model to improve the performance. The performance can be evaluated with classification accuracy or Area Under ROC Curve (AUC). 


In [ ]:
%conda install -c conda-forge tensorflow=2.10.0
%conda install -c conda-forge tensorflow-quantum=0.3.0


Channels:
 - conda-forge
Platform: osx-arm64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 24.7.1
    latest version: 25.1.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
Channels:
 - conda-forge
Platform: osx-arm64
Solving environment: failed

PackagesNotFoundError: The following packages are not available from current channels:

  - tensorflow-quantum=0.3.0*

Current channels:

  - https://conda.anaconda.org/conda-forge

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.
  Using cached tf_keras-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorflow-2.19.0-cp39-cp39-macosx_12_0_arm64.

In [3]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"
import tensorflow as tf
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [5]:

# Load the file with pickle support
data = np.load('./QIS_EXAM_200Events.npz', allow_pickle=True)

# Print keys
print("Keys in file:", list(data.keys()))

# Examine each key
for key in data.keys():
    arr = data[key]
    if isinstance(arr, np.ndarray):
        print(f"{key}: {type(arr)} Shape: {arr.shape}, Dtype: {arr.dtype}")
    else:
        print(f"{key}: {type(arr)}")
        
    # Try to print first few values
    try:
        if isinstance(arr, np.ndarray) and arr.size > 0:
            print("First few values:", arr.flatten()[:5])
        else:
            print("Value:", arr)
    except:
        print("Cannot display values")

Keys in file: ['training_input', 'test_input']
training_input: <class 'numpy.ndarray'> Shape: (), Dtype: object
First few values: [{'0': array([[-0.43079088,  0.86834819, -0.92614721, -0.92662029, -0.56900862],
        [ 0.33924198,  0.56155499,  0.93097459, -0.91631726, -0.54463516],
        [-0.42888879,  0.87064961, -0.92782179, -0.77533991, -0.58329176],
        [-0.43262871,  0.86128919, -0.92240878, -0.88048862, -0.49963115],
        [-0.99925345, -0.99949586,  0.07753685, -0.84218034, -0.5149399 ],
        [-0.99631106, -0.99775978,  0.0756427 , -0.54117216, -0.66299335],
        [-0.42645921,  0.87141204, -0.92908723, -0.52650143, -0.62187526],
        [ 0.34317906,  0.57125045,  0.92638556, -0.85113425, -0.40170562],
        [-0.99904849, -0.99933931,  0.07737929, -0.81161066, -0.53550246],
        [ 0.3371327 ,  0.55874622,  0.92996976, -0.9117092 , -0.50996097],
        [ 0.89649306, -0.95523176, -0.66298651, -0.71276678, -0.62698893],
        [ 0.34293232,  0.56408047,  0.9

In [6]:

# Data loading and processing
def load_and_process_data(file_path):
    """
    Simplified extraction based on the observed structure
    """
    # Load the data
    data = np.load(file_path, allow_pickle=True)
    
    # Extract the training features from the dictionary
    training_dict = data['training_input'].item()  # Get the dictionary from the array
    training_key = list(training_dict.keys())[0]  # Get the first key ('0')
    X_train = training_dict[training_key]  # Get the feature array
    
    # Extract the test features from the dictionary
    test_dict = data['test_input'].item()  # Get the dictionary from the array
    test_key = list(test_dict.keys())[0]  # Get the first key ('0')
    X_test = test_dict[test_key]  # Get the feature array
    
    print(f"Training features shape: {X_train.shape}")
    print(f"Test features shape: {X_test.shape}")
    
    # Create labels (assuming we need to create binary labels)
    # Method 1: Using principal component
    from sklearn.decomposition import PCA
    
    pca = PCA(n_components=1)
    train_pca = pca.fit_transform(X_train)
    test_pca = pca.transform(X_test)
    
    # Use median as threshold for binary labels
    train_threshold = np.median(train_pca)
    y_train = (train_pca > train_threshold).astype(int).ravel()
    y_test = (test_pca > train_threshold).astype(int).ravel()
    
    print(f"Created labels - Training: {np.sum(y_train)} signal events, {len(y_train) - np.sum(y_train)} background events")
    print(f"Created labels - Test: {np.sum(y_test)} signal events, {len(y_test) - np.sum(y_test)} background events")
    
    # Scale the features to [0, π] for quantum circuit
    scaler = MinMaxScaler(feature_range=(0, np.pi))
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    return X_train_scaled, X_test_scaled, y_train, y_test, scaler

# Dimension reduction if needed
def apply_pca(X_train, X_test, n_components=None):
    """Apply PCA dimension reduction if the feature dimension is too large"""
    from sklearn.decomposition import PCA
    
    # Determine optimal number of components to retain 95% variance
    if n_components is None:
        pca = PCA(n_components=0.95)
        pca.fit(X_train)
        n_components = pca.n_components_
    else:
        pca = PCA(n_components=n_components)
        pca.fit(X_train)
    
    print(f"Reducing dimensions from {X_train.shape[1]} to {n_components} with PCA")
    print(f"Explained variance ratio: {sum(pca.explained_variance_ratio_):.4f}")
    
    X_train_pca = pca.transform(X_train)
    X_test_pca = pca.transform(X_test)
    
    return X_train_pca, X_test_pca, pca

# Define quantum devices
def setup_quantum_circuit(n_qubits):
    """Setup quantum circuit devices for generator and discriminator"""
    print(f"Setting up quantum circuits with {n_qubits} qubits")
    
    # Create PennyLane device for generator
    dev_gen = qml.device("default.qubit", wires=n_qubits)
    
    # Create PennyLane device for discriminator
    dev_disc = qml.device("default.qubit", wires=n_qubits)
    
    # Define the quantum circuit for the generator
    @qml.qnode(dev_gen)
    def quantum_generator(noise, weights):
        """Variational quantum circuit for generator"""
        # Initialize with Hadamard gates
        for i in range(n_qubits):
            qml.Hadamard(wires=i)
        
        # Encode input noise
        for i in range(n_qubits):
            qml.RY(noise[i], wires=i)
        
        # Variational layers
        n_layers = len(weights) // (2 * n_qubits)
        for l in range(n_layers):
            # Rotation layer
            for i in range(n_qubits):
                qml.RX(weights[l*2*n_qubits + i], wires=i)
                qml.RZ(weights[l*2*n_qubits + n_qubits + i], wires=i)
            
            # Entanglement layer
            for i in range(n_qubits-1):
                qml.CNOT(wires=[i, i+1])
        
        # Return expectation values
        return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
    
    # Define the quantum circuit for the discriminator
    @qml.qnode(dev_disc)
    def quantum_discriminator(inputs, weights):
        """Quantum circuit for feature extraction in the discriminator"""
        # Encode input data
        for i in range(n_qubits):
            qml.RX(inputs[i], wires=i)
        
        # Entanglement layer
        for i in range(n_qubits-1):
            qml.CNOT(wires=[i, i+1])
        
        # Rotation layer with trainable weights
        for i in range(n_qubits):
            qml.RY(weights[i], wires=i)
        
        # Return expectation values
        return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
    
    return quantum_generator, quantum_discriminator

# Build the generator model
class QuantumGenerator(tf.keras.Model):
    def __init__(self, n_qubits, n_layers=2):
        super(QuantumGenerator, self).__init__()
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        
        # Set up quantum circuit
        self.quantum_generator, _ = setup_quantum_circuit(n_qubits)
        
        # Classical pre-processing
        self.dense = tf.keras.layers.Dense(n_qubits, activation='tanh')
        
        # Initialize weights for quantum circuit
        weight_shapes = {"weights": (2 * n_qubits * n_layers,)}
        self.qlayer = qml.qnn.KerasLayer(self.quantum_generator, weight_shapes, output_dim=n_qubits)
        
    def call(self, inputs):
        x = self.dense(inputs)
        return self.qlayer(x)

# Build the discriminator model
class QuantumDiscriminator(tf.keras.Model):
    def __init__(self, n_qubits):
        super(QuantumDiscriminator, self).__init__()
        self.n_qubits = n_qubits
        
        # Set up quantum circuit
        _, self.quantum_discriminator = setup_quantum_circuit(n_qubits)
        
        # Initialize weights for quantum circuit
        weight_shapes = {"weights": (n_qubits,)}
        self.qlayer = qml.qnn.KerasLayer(self.quantum_discriminator, weight_shapes, output_dim=n_qubits)
        
        # Classical post-processing
        self.dense1 = tf.keras.layers.Dense(8, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')
        
    def call(self, inputs):
        x = self.qlayer(inputs)
        x = self.dense1(x)
        return self.dense2(x)

# Build the GAN
class QGAN(tf.keras.Model):
    def __init__(self, generator, discriminator):
        super(QGAN, self).__init__()
        self.generator = generator
        self.discriminator = discriminator
        
    def call(self, inputs):
        generated = self.generator(inputs)
        return self.discriminator(generated)

# Train the GAN
def train_qgan(generator, discriminator, gan, X_train, y_train, epochs=50, batch_size=16):
    # Define optimizers
    generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    
    # Define loss function
    loss_fn = tf.keras.losses.BinaryCrossentropy()
    
    # Prepare real signal data
    signal_data = X_train[y_train == 1]
    background_data = X_train[y_train == 0]
    
    print(f"Training with {signal_data.shape[0]} signal events and {background_data.shape[0]} background events")
    
    history = {'disc_loss': [], 'gen_loss': []}
    
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        
        # Train discriminator
        with tf.GradientTape() as disc_tape:
            # Sample real signal data
            real_indices = np.random.randint(0, signal_data.shape[0], batch_size//2)
            real_signal = signal_data[real_indices]
            real_labels = np.ones((batch_size//2, 1))
            
            # Sample real background data
            bg_indices = np.random.randint(0, background_data.shape[0], batch_size//2)
            real_bg = background_data[bg_indices]
            bg_labels = np.zeros((batch_size//2, 1))
            
            # Combine real datasets
            real_samples = np.vstack([real_signal, real_bg])
            real_labels = np.vstack([real_labels, bg_labels])
            
            # Generate fake signal data
            noise = np.random.normal(0, 1, (batch_size, generator.n_qubits))
            fake_samples = generator(noise)
            fake_labels = np.zeros((batch_size, 1))
            
            # Forward pass
            real_predictions = discriminator(real_samples)
            fake_predictions = discriminator(fake_samples)
            
            # Calculate loss
            real_loss = loss_fn(real_labels, real_predictions)
            fake_loss = loss_fn(fake_labels, fake_predictions)
            disc_loss = real_loss + fake_loss
        
        # Update discriminator
        disc_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
        discriminator_optimizer.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))
        
        # Train generator
        with tf.GradientTape() as gen_tape:
            # Generate fake samples
            noise = np.random.normal(0, 1, (batch_size, generator.n_qubits))
            fake_samples = generator(noise)
            
            # Get discriminator predictions
            predictions = discriminator(fake_samples)
            
            # Use real labels for generator loss (trying to fool discriminator)
            gen_loss = loss_fn(np.ones((batch_size, 1)), predictions)
        
        # Update generator
        gen_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
        generator_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))
        
        history['disc_loss'].append(float(disc_loss))
        history['gen_loss'].append(float(gen_loss))
        
        if (epoch + 1) % 5 == 0:
            print(f"Discriminator Loss: {float(disc_loss):.4f}, Generator Loss: {float(gen_loss):.4f}")
    
    # Plot training history
    plt.figure(figsize=(10, 6))
    plt.plot(history['disc_loss'], label='Discriminator Loss')
    plt.plot(history['gen_loss'], label='Generator Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('QGAN Training History')
    plt.legend()
    plt.show()
    
    return history

# Evaluate the model
def evaluate_model(discriminator, X_test, y_test):
    """Evaluate the model on test data"""
    y_pred = discriminator.predict(X_test)
    
    # Calculate ROC curve and AUC
    fpr, tpr, _ = roc_curve(y_test, y_pred)
    roc_auc = auc(fpr, tpr)
    
    # Calculate accuracy
    y_pred_binary = (y_pred > 0.5).astype(int)
    accuracy = np.mean(y_pred_binary.flatten() == y_test)
    
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test AUC: {roc_auc:.4f}")
    
    # Plot ROC curve
    plt.figure(figsize=(10, 8))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # Confusion matrix
    from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
    cm = confusion_matrix(y_test, y_pred_binary)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Background', 'Signal'])
    disp.plot(cmap='Blues')
    plt.title('Confusion Matrix')
    plt.show()
    
    return accuracy, roc_auc

# Hyperparameter tuning
def hyperparameter_tuning(X_train, y_train, X_test, y_test):
    """Simple hyperparameter tuning"""
    best_auc = 0
    best_params = {}
    results = []
    
    # Define hyperparameter space
    n_layers_options = [1, 2]
    learning_rates = [0.0001, 0.001]
    batch_sizes = [8, 16]
    
    for n_layers in n_layers_options:
        for lr in learning_rates:
            for batch_size in batch_sizes:
                print(f"Testing: n_layers={n_layers}, lr={lr}, batch_size={batch_size}")
                
                # Initialize models
                generator = QuantumGenerator(n_qubits=X_train.shape[1], n_layers=n_layers)
                discriminator = QuantumDiscriminator(n_qubits=X_train.shape[1])
                gan = QGAN(generator, discriminator)
                
                # Compile models
                discriminator.compile(
                    optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                    loss='binary_crosssentropy',
                    metrics=['accuracy']
                )
                
                gan.compile(
                    optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                    loss='binary_crosssentropy'
                )
                
                # Train
                history = train_qgan(
                    generator, discriminator, gan,
                    X_train, y_train,
                    epochs=15,  # Reduced for hyperparameter search
                    batch_size=batch_size
                )
                
                # Evaluate
                accuracy, auc_score = evaluate_model(discriminator, X_test, y_test)
                
                # Store result
                results.append({
                    'n_layers': n_layers,
                    'learning_rate': lr,
                    'batch_size': batch_size,
                    'accuracy': accuracy,
                    'auc': auc_score
                })
                
                # Check if better
                if auc_score > best_auc:
                    best_auc = auc_score
                    best_params = {
                        'n_layers': n_layers,
                        'learning_rate': lr,
                        'batch_size': batch_size
                    }
    
    print(f"Best parameters: {best_params}")
    print(f"Best AUC: {best_auc:.4f}")
    
    # Plot hyperparameter results
    plt.figure(figsize=(12, 6))
    x = range(len(results))
    plt.bar(x, [r['auc'] for r in results], alpha=0.6, label='AUC')
    plt.xticks(x, [f"L{r['n_layers']},LR{r['learning_rate']},B{r['batch_size']}" for r in results], rotation=90)
    plt.xlabel('Hyperparameters')
    plt.ylabel('AUC Score')
    plt.title('Hyperparameter Tuning Results')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    return best_params, results


In [4]:

import os
# Set environment variable to use legacy Keras (Keras 2) with TensorFlow
os.environ["TF_USE_LEGACY_KERAS"] = "1"

# Import tf_keras instead of keras
import tensorflow as tf
import tf_keras as keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import sympy
import cirq
import tensorflow_quantum as tfq

def main():
    print("Loading and processing data...")
    try:
        X_train, X_test, y_train, y_test, _ = load_and_process_data('./QIS_EXAM_200Events.npz')
        
        # Check feature dimensions - quantum circuits work better with fewer qubits
        n_features = X_train.shape[1]
        if n_features > 8:
            print(f"High feature dimension detected ({n_features}). Applying PCA...")
            from sklearn.decomposition import PCA
            pca = PCA(n_components=8)
            X_train = pca.fit_transform(X_train)
            X_test = pca.transform(X_test)
            n_features = X_train.shape[1]
        
        print(f"Building quantum models with {n_features} features/qubits")
        
        # Initialize models using the function-based approach
        generator = build_generator(n_qubits=n_features, n_layers=2)
        discriminator = build_discriminator(n_qubits=n_features)
        
        # Build GAN
        gan = build_qgan(generator, discriminator)
        
        # Compile models
        discriminator.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        gan.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss='binary_crossentropy'
        )
        
        # Training
        print("Training QGAN...")
        history = train_qgan(generator, discriminator, gan, X_train, y_train, epochs=50, batch_size=16)
        
        # Evaluation
        print("Evaluating model...")
        accuracy, auc_score = evaluate_model(discriminator, X_test, y_test)
        
        # Optional: Uncomment to run hyperparameter tuning
        run_tuning = input("Do you want to run hyperparameter tuning? (y/n): ").lower() == 'y'
        if run_tuning:
            print("Running hyperparameter tuning...")
            best_params = tune_hyperparameters(X_train, y_train, X_test, y_test, n_features)
            
            # Train final model with best params
            print(f"Training final model with best parameters: {best_params}")
            generator = build_generator(n_qubits=n_features, n_layers=best_params['n_layers'])
            discriminator = build_discriminator(n_qubits=n_features)
            gan = build_qgan(generator, discriminator)
            
            discriminator.compile(
                optimizer=keras.optimizers.Adam(learning_rate=best_params['lr']),
                loss='binary_crossentropy',
                metrics=['accuracy']
            )
            
            gan.compile(
                optimizer=keras.optimizers.Adam(learning_rate=best_params['lr']),
                loss='binary_crossentropy'
            )
            
            history = train_qgan(
                generator, discriminator, gan,
                X_train, y_train,
                epochs=50,
                batch_size=best_params['batch_size']
            )
            
            print("Evaluating final model...")
            accuracy, auc_score = evaluate_model(discriminator, X_test, y_test)
        
        print("QGAN implementation complete")
        return accuracy, auc_score
        
    except Exception as e:
        print(f"Error in main execution: {e}")
        import traceback
        traceback.print_exc()

# Simplified code to extract and process data from your structure
def extract_data_simple(file_path):
    """
    Simplified extraction based on the observed structure
    """
    # Load the data
    data = np.load(file_path, allow_pickle=True)
    
    # Extract the training features from the dictionary
    training_dict = data['training_input'].item()  # Get the dictionary from the array
    training_key = list(training_dict.keys())[0]  # Get the first key ('0')
    X_train = training_dict[training_key]  # Get the feature array
    
    # Extract the test features from the dictionary
    test_dict = data['test_input'].item()  # Get the dictionary from the array
    test_key = list(test_dict.keys())[0]  # Get the first key ('0')
    X_test = test_dict[test_key]  # Get the feature array
    
    print(f"Training features shape: {X_train.shape}")
    print(f"Test features shape: {X_test.shape}")
    
    # Create labels (assuming we need to create binary labels)
    # Method 1: Using principal component
    from sklearn.decomposition import PCA
    
    pca = PCA(n_components=1)
    train_pca = pca.fit_transform(X_train)
    test_pca = pca.transform(X_test)
    
    # Use median as threshold for binary labels
    train_threshold = np.median(train_pca)
    y_train = (train_pca > train_threshold).astype(int).ravel()
    y_test = (test_pca > train_threshold).astype(int).ravel()
    
    print(f"Created labels - Training: {np.sum(y_train)} signal events, {len(y_train) - np.sum(y_train)} background events")
    print(f"Created labels - Test: {np.sum(y_test)} signal events, {len(y_test) - np.sum(y_test)} background events")
    
    # Scale the features to [0, π] for quantum circuit
    scaler = MinMaxScaler(feature_range=(0, np.pi))
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    return X_train_scaled, X_test_scaled, y_train, y_test, scaler

# Quantum Circuit Setup for TFQ
def create_quantum_circuits(n_qubits, n_layers=2):
    """Create quantum circuits for the QGAN"""
    # Create qubits
    qubits = cirq.GridQubit.rect(1, n_qubits)
    
    # Create generator circuit
    gen_circuit = cirq.Circuit()
    # Initial Hadamard layer
    gen_circuit.append(cirq.H.on_each(qubits))
    
    # Variational layers
    for l in range(n_layers):
        # Rotation layer
        for i, q in enumerate(qubits):
            gen_circuit.append(cirq.rx(sympy.Symbol(f'rx_{l}_{i}'))(q))
            gen_circuit.append(cirq.rz(sympy.Symbol(f'rz_{l}_{i}'))(q))
        
        # Entanglement layer
        for i in range(n_qubits-1):
            gen_circuit.append(cirq.CNOT(qubits[i], qubits[i+1]))
    
    # Create discriminator circuit
    disc_circuit = cirq.Circuit()
    
    # Data encoding layer
    for i, q in enumerate(qubits):
        disc_circuit.append(cirq.rx(sympy.Symbol(f'x_{i}'))(q))
    
    # Entanglement layer
    for i in range(n_qubits-1):
        disc_circuit.append(cirq.CNOT(qubits[i], qubits[i+1]))
    
    # Rotation layer
    for i, q in enumerate(qubits):
        disc_circuit.append(cirq.ry(sympy.Symbol(f'ry_{i}'))(q))
    
    return gen_circuit, disc_circuit, qubits

# Build generator model
def build_generator(n_qubits, n_layers=2):
    """Build the generator model"""
    # Create input layer for noise
    noise_input = keras.layers.Input(shape=(n_qubits,))
    
    # Dense layer to preprocess noise
    x = keras.layers.Dense(n_qubits, activation='tanh')(noise_input)
    
    # Create the quantum circuit
    gen_circuit, _, qubits = create_quantum_circuits(n_qubits, n_layers)
    
    # Create parameter symbols for the circuit
    params = []
    for l in range(n_layers):
        for i in range(n_qubits):
            params.append(sympy.Symbol(f'rx_{l}_{i}'))
            params.append(sympy.Symbol(f'rz_{l}_{i}'))
    
    # Create a PQC layer
    # First, convert the circuit to a TFQ object and set up readout operators
    readout_operators = [cirq.Z(q) for q in qubits]
    expectation_layer = tfq.layers.ControlledPQC(
        gen_circuit, readout_operators, 
        initializer=tf.keras.initializers.RandomUniform(0, 2*np.pi)
    )
    
    # Connect inputs to the quantum circuit
    quantum_output = expectation_layer([x, tf.zeros_like(x)])
    
    # Return the generator model
    return keras.Model(inputs=noise_input, outputs=quantum_output, name="Generator")

# Build discriminator model
def build_discriminator(n_qubits):
    """Build the discriminator model"""
    # Create input layer for features
    feature_input = keras.layers.Input(shape=(n_qubits,))
    
    # Create the quantum circuit
    _, disc_circuit, qubits = create_quantum_circuits(n_qubits)
    
    # Create parameter symbols for the circuit
    params = [sympy.Symbol(f'ry_{i}') for i in range(n_qubits)]
    
    # Create a PQC layer
    readout_operators = [cirq.Z(q) for q in qubits]
    expectation_layer = tfq.layers.PQC(
        disc_circuit, readout_operators,
        initializer=tf.keras.initializers.RandomUniform(0, 2*np.pi)
    )
    
    # Connect inputs to the quantum circuit
    quantum_output = expectation_layer(feature_input)
    
    # Add classical post-processing
    x = keras.layers.Dense(8, activation='relu')(quantum_output)
    output = keras.layers.Dense(1, activation='sigmoid')(x)
    
    # Return the discriminator model
    return keras.Model(inputs=feature_input, outputs=output, name="Discriminator")

# Build the QGAN model
def build_qgan(generator, discriminator):
    """Build the complete GAN model"""
    # For the combined model, we only train the generator
    discriminator.trainable = False
    
    # The generator takes noise as input and generates fake samples
    z = keras.layers.Input(shape=(generator.input_shape[1],))
    fake_samples = generator(z)
    
    # The discriminator determines the validity of the fake samples
    validity = discriminator(fake_samples)
    
    # The combined model (generator + discriminator)
    return keras.Model(inputs=z, outputs=validity, name="QGAN")

# Train the QGAN
def train_qgan(generator, discriminator, gan, X_train, y_train, epochs=50, batch_size=16):
    # Prepare real signal and background data
    signal_data = X_train[y_train == 1]
    background_data = X_train[y_train == 0]
    
    # Training loop
    d_losses = []
    g_losses = []
    
    for epoch in range(epochs):
        # Train discriminator
        # Select real samples
        signal_idx = np.random.randint(0, signal_data.shape[0], batch_size // 2)
        bg_idx = np.random.randint(0, background_data.shape[0], batch_size // 2)
        
        real_signal = signal_data[signal_idx]
        real_bg = background_data[bg_idx]
        
        real_samples = np.vstack([real_signal, real_bg])
        real_labels = np.vstack([
            np.ones((batch_size // 2, 1)),
            np.zeros((batch_size // 2, 1))
        ])
        
        # Generate fake samples
        noise = np.random.normal(0, 1, (batch_size, generator.input_shape[1]))
        fake_samples = generator.predict(noise)
        fake_labels = np.zeros((batch_size, 1))
        
        # Train the discriminator
        d_loss_real = discriminator.train_on_batch(real_samples, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_samples, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # Train generator
        noise = np.random.normal(0, 1, (batch_size, generator.input_shape[1]))
        valid_labels = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, valid_labels)
        
        # Store losses
        d_losses.append(d_loss[0])
        g_losses.append(g_loss)
        
        # Print progress
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs} | D Loss: {d_loss[0]:.4f} | G Loss: {g_loss:.4f}")
    
    # Plot loss history
    plt.figure(figsize=(10, 5))
    plt.plot(d_losses, label='Discriminator')
    plt.plot(g_losses, label='Generator')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('QGAN Training Loss')
    plt.show()
    
    return {'d_loss': d_losses, 'g_loss': g_losses}

# Evaluate the model
def evaluate_model(discriminator, X_test, y_test):
    # Get predictions
    y_pred = discriminator.predict(X_test)
    
    # Calculate ROC curve and AUC
    fpr, tpr, _ = roc_curve(y_test, y_pred)
    roc_auc = auc(fpr, tpr)
    
    # Calculate accuracy
    y_pred_binary = (y_pred > 0.5).astype(int)
    accuracy = np.mean(y_pred_binary.flatten() == y_test)
    
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test AUC: {roc_auc:.4f}")
    
    # Plot ROC curve
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()
    
    return accuracy, roc_auc

# Hyperparameter tuning
def tune_hyperparameters(X_train, y_train, X_test, y_test, n_qubits):
    results = []
    
    # Hyperparameters to test
    n_layers_options = [1, 2]
    learning_rates = [0.001, 0.01]
    batch_sizes = [8, 16]
    
    best_auc = 0
    best_params = {}
    
    for n_layers in n_layers_options:
        for lr in learning_rates:
            for batch_size in batch_sizes:
                print(f"Testing: n_layers={n_layers}, lr={lr}, batch_size={batch_size}")
                
                # Build models
                generator = build_generator(n_qubits, n_layers)
                discriminator = build_discriminator(n_qubits)
                
                # Compile models
                discriminator.compile(
                    optimizer=keras.optimizers.Adam(learning_rate=lr),
                    loss='binary_crossentropy',
                    metrics=['accuracy']
                )
                
                gan = build_qgan(generator, discriminator)
                gan.compile(
                    optimizer=keras.optimizers.Adam(learning_rate=lr),
                    loss='binary_crossentropy'
                )
                
                # Train with reduced epochs for tuning
                train_qgan(generator, discriminator, gan, X_train, y_train, 
                          epochs=20, batch_size=batch_size)
                
                # Evaluate
                _, auc_score = evaluate_model(discriminator, X_test, y_test)
                
                # Record results
                results.append({
                    'n_layers': n_layers,
                    'lr': lr,
                    'batch_size': batch_size,
                    'auc': auc_score
                })
                
                # Update best parameters
                if auc_score > best_auc:
                    best_auc = auc_score
                    best_params = {
                        'n_layers': n_layers,
                        'lr': lr,
                        'batch_size': batch_size
                    }
    
    print(f"Best parameters: {best_params}")
    print(f"Best AUC: {best_auc:.4f}")
    
    return best_params

if __name__ == "__main__":
    # You might need to install these packages:
    # !pip install tf_keras tensorflow_quantum sympy cirq
    
    try:
        main()
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
        
        print("\nIf you're encountering dependency issues, try installing the required packages:")
        print("pip install tf_keras tensorflow_quantum cirq sympy")

if __name__ == "__main__":
    main()

2025-03-27 01:45:12.045309: E tensorflow/core/lib/monitoring/collection_registry.cc:81] Cannot register 2 metrics with the same name: /tensorflow/api/enable_tensor_equality


AlreadyExistsError: Another metric with the same name already exists.